# 🎬 ComfyUI + Wan2.1 — Geração de Vídeo por IA no Google Colab

**Modelos:** Wan2.1-T2V-1.3B (Texto → Vídeo) + Wan2.1-I2V-1.3B (Imagem → Vídeo)
**GPU:** T4 (15GB VRAM) — Colab Free
**Framework:** ComfyUI (sem filtros NSFW)

### Requisitos de VRAM
| Componente | Formato | Tamanho |
|---|---|---|
| Diffusion Model T2V 1.3B | FP16 | ~2.6 GB |
| Diffusion Model I2V 1.3B | FP16 | ~2.6 GB |
| Text Encoder UMT5-XXL | FP8 | ~5 GB |
| CLIP Vision H (I2V) | FP16 | ~3.9 GB |
| VAE | FP32 | ~300 MB |
| **Total T2V** | | **~8 GB** |
| **Total T2V + I2V** | | **~14.5 GB** |

### Limites no T4
- Resolução máxima: **480×320**
- Duração: **2-4 segundos** (33-65 frames)
- Tempo de geração: **10-30 min** por clip
- Frames devem ser `4n+1`: 5, 9, 13, 17, 21, 25, 29, 33, 49, 65...

> **Instruções:** Execute as células em ordem (1→5). Célula 6 = T2V, Célula 7 = I2V.
> Para I2V, execute também a célula 4b para baixar os modelos extra.

## 1️⃣ Montar Google Drive (Opcional)
Ative `Use_Gdrive = True` para salvar os modelos (~8GB) no Google Drive e evitar re-baixar a cada sessão.

In [1]:
#@title 1. Montar Google Drive (Opcional)
#@markdown Ative para persistir modelos entre sessões (~8GB salvos no Drive).

Use_Gdrive = False  #@param {type:"boolean"}

import os

COMFYUI_PATH = "/content/ComfyUI"
GDRIVE_MODELS = "/content/gdrive/MyDrive/ComfyUI/models"

# Diretórios de modelos do ComfyUI
MODEL_DIRS = {
    "diffusion_models": f"{COMFYUI_PATH}/models/diffusion_models",
    "text_encoders":    f"{COMFYUI_PATH}/models/text_encoders",
    "vae":              f"{COMFYUI_PATH}/models/vae",
}

if Use_Gdrive:
    from google.colab import drive
    drive.mount('/content/gdrive')
    # Criar diretórios no GDrive para persistir modelos
    for subdir in MODEL_DIRS:
        gdrive_dir = f"{GDRIVE_MODELS}/{subdir}"
        os.makedirs(gdrive_dir, exist_ok=True)
    print("✅ Google Drive montado! Modelos serão salvos em:", GDRIVE_MODELS)
else:
    print("ℹ️  Usando armazenamento local do Colab (modelos serão perdidos ao desconectar).")

ℹ️  Usando armazenamento local do Colab (modelos serão perdidos ao desconectar).


## 2️⃣ Instalar ComfyUI
Clona o repositório ComfyUI e instala dependências. Se GDrive estiver ativo, cria symlinks dos diretórios de modelos para o Drive.

In [ ]:
#@title 2. Instalar ComfyUI
import os, shutil, subprocess

COMFYUI_PATH = "/content/ComfyUI"
GDRIVE_MODELS = "/content/gdrive/MyDrive/ComfyUI/models"

# --- Clonar ComfyUI ---
if not os.path.exists(COMFYUI_PATH):
    !git clone https://github.com/comfyanonymous/ComfyUI.git {COMFYUI_PATH}
    print("✅ ComfyUI clonado.")
else:
    print("ℹ️  ComfyUI já existe, atualizando...")
    %cd {COMFYUI_PATH}
    !git pull

# --- Instalar dependências ---
%cd {COMFYUI_PATH}
!pip install -q -r requirements.txt
print("✅ Dependências instaladas.")

# --- Symlinks para GDrive (se ativo) ---
if Use_Gdrive and os.path.exists(GDRIVE_MODELS):
    MODEL_DIRS = {
        "diffusion_models": f"{COMFYUI_PATH}/models/diffusion_models",
        "text_encoders":    f"{COMFYUI_PATH}/models/text_encoders",
        "vae":              f"{COMFYUI_PATH}/models/vae",
    }
    for subdir, local_dir in MODEL_DIRS.items():
        gdrive_dir = f"{GDRIVE_MODELS}/{subdir}"
        if os.path.islink(local_dir) and os.readlink(local_dir) == gdrive_dir:
            continue
        if os.path.exists(local_dir) and not os.path.islink(local_dir):
            for f in os.listdir(local_dir):
                src = os.path.join(local_dir, f)
                dst = os.path.join(gdrive_dir, f)
                if not os.path.exists(dst):
                    shutil.move(src, dst)
            shutil.rmtree(local_dir)
        elif os.path.islink(local_dir):
            os.unlink(local_dir)
        os.symlink(gdrive_dir, local_dir)
    print("✅ Symlinks criados → modelos salvos no Google Drive.")

# --- Verificar GPU ---
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"✅ GPU: {gpu_name} | VRAM: {vram:.1f} GB | PyTorch: {torch.__version__} | CUDA: {torch.version.cuda}")
else:
    print("⚠️  GPU não detectada! Vá em Runtime → Change runtime type → GPU (T4).")

## 3️⃣ Instalar Custom Nodes
- **ComfyUI-WanVideoWrapper** (kijai) — Controle avançado de Wan2.1, suporte LoRA, GGUF, Image-to-Video
- **ComfyUI-GGUF** (city96) — Carregar modelos em formato GGUF quantizado
- **ComfyUI-VideoHelperSuite** (Kosinkadink) — Nó `VideoCombine` para salvar output em **MP4/GIF** diretamente

> **Nota:** O WanVideoWrapper é instalado com `--no-deps` para evitar downgrade do PyTorch do Colab. As dependências críticas são instaladas manualmente.

In [ ]:
#@title 3. Instalar Custom Nodes (WanVideoWrapper + GGUF + VideoHelperSuite)
import os

COMFYUI_PATH = "/content/ComfyUI"
CUSTOM_NODES = f"{COMFYUI_PATH}/custom_nodes"

# Lista de custom nodes para instalar
nodes = {
    "ComfyUI-WanVideoWrapper":  "https://github.com/kijai/ComfyUI-WanVideoWrapper",
    "ComfyUI-GGUF":             "https://github.com/city96/ComfyUI-GGUF",
    "ComfyUI-VideoHelperSuite": "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite",
    "ComfyUI_IPAdapter_plus":   "https://github.com/cubiq/ComfyUI_IPAdapter_plus",
}

# Nodes que precisam de --no-deps (para não fazer downgrade do PyTorch do Colab)
NO_DEPS_NODES = {"ComfyUI-WanVideoWrapper"}

for name, url in nodes.items():
    node_path = f"{CUSTOM_NODES}/{name}"
    if os.path.exists(node_path):
        print(f"ℹ️  {name} já existe, atualizando...")
        !cd {node_path} && git pull -q
    else:
        !git clone {url} {node_path}

    # Instalar requirements
    req_file = f"{node_path}/requirements.txt"
    if os.path.exists(req_file):
        if name in NO_DEPS_NODES:
            # --no-deps evita que pip tente fazer downgrade do Torch otimizado do Colab
            !pip install -q -r {req_file} --no-deps
        else:
            !pip install -q -r {req_file}
    print(f"✅ {name} instalado.")

# Instalar dependências críticas do WanVideoWrapper manualmente
# (sem arrastar versões conflitantes de torch/numpy)
print("\n📦 Instalando dependências críticas manualmente...")
!pip install -q diffusers accelerate transformers sentencepiece protobuf
print("✅ Dependências críticas instaladas.")

# Verificar ffmpeg (necessário para VideoHelperSuite salvar MP4)
import shutil
if shutil.which("ffmpeg"):
    print("✅ ffmpeg encontrado.")
else:
    print("⬇️  Instalando ffmpeg...")
    !apt-get install -y -qq ffmpeg > /dev/null 2>&1
    print("✅ ffmpeg instalado.")

# Listar nodes instalados
print("\n📦 Custom nodes instalados:")
for d in sorted(os.listdir(CUSTOM_NODES)):
    full = os.path.join(CUSTOM_NODES, d)
    if os.path.isdir(full) and not d.startswith('.'):
        print(f"   • {d}")

## 4️⃣ Baixar Modelos Wan2.1 (T2V-1.3B)
Downloads do repositório [Comfy-Org/Wan_2.1_ComfyUI_repackaged](https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged):

| Ficheiro | Formato | Tamanho | Destino |
|---|---|---|---|
| `wan2.1_t2v_1.3B_fp16.safetensors` | FP16 | ~2.6 GB | `models/diffusion_models/` |
| `umt5_xxl_fp8_e4m3fn_scaled.safetensors` | FP8 | ~5 GB | `models/text_encoders/` |
| `wan_2.1_vae.safetensors` | FP32 | ~300 MB | `models/vae/` |

> Se GDrive está ativo, os ficheiros são salvos no Drive via symlink (não precisará baixar novamente).

In [ ]:
#@title 4. Baixar Modelos Wan2.1 (~8GB total)
import os

COMFYUI_PATH = "/content/ComfyUI"
HF_BASE = "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files"

# Modelos para baixar: (url_relativo, pasta_destino, nome_ficheiro)
MODELS = [
    (
        f"{HF_BASE}/diffusion_models/wan2.1_t2v_1.3B_fp16.safetensors",
        f"{COMFYUI_PATH}/models/diffusion_models",
        "wan2.1_t2v_1.3B_fp16.safetensors",
    ),
    (
        f"{HF_BASE}/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors",
        f"{COMFYUI_PATH}/models/text_encoders",
        "umt5_xxl_fp8_e4m3fn_scaled.safetensors",
    ),
    (
        f"{HF_BASE}/vae/wan_2.1_vae.safetensors",
        f"{COMFYUI_PATH}/models/vae",
        "wan_2.1_vae.safetensors",
    ),
]

def format_size(size_bytes):
    """Formatar bytes para leitura humana."""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.1f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.1f} TB"

total_downloaded = 0

for url, dest_dir, filename in MODELS:
    filepath = os.path.join(dest_dir, filename)
    os.makedirs(dest_dir, exist_ok=True)

    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        print(f"✅ {filename} já existe ({format_size(size)}) — pulando download.")
    else:
        print(f"⬇️  Baixando {filename}...")
        !wget -q --show-progress -O "{filepath}" "{url}"
        if os.path.exists(filepath):
            size = os.path.getsize(filepath)
            total_downloaded += size
            print(f"   ✅ Concluído: {format_size(size)}")
        else:
            print(f"   ❌ ERRO: Download falhou para {filename}!")

# Resumo
print("\n" + "="*50)
print("📊 Resumo dos modelos:")
for url, dest_dir, filename in MODELS:
    filepath = os.path.join(dest_dir, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        print(f"   ✅ {filename}: {format_size(size)}")
    else:
        print(f"   ❌ {filename}: NÃO ENCONTRADO")
if total_downloaded > 0:
    print(f"\n   Total baixado nesta sessão: {format_size(total_downloaded)}")
print("="*50)

## 4b️⃣ Baixar Modelos Extra para Image-to-Video (Opcional)
Se quiser usar **Imagem → Vídeo** (I2V), precisa de 2 modelos adicionais:

| Ficheiro | Formato | Tamanho | Destino |
|---|---|---|---|
| `wan2.1_i2v_480p_1.3B_fp16.safetensors` | FP16 | ~2.6 GB | `models/diffusion_models/` |
| `clip_vision_h.safetensors` | FP16 | ~3.9 GB | `models/clip_vision/` |

> **Nota:** O text encoder (UMT5-XXL) e o VAE são partilhados com o T2V — não precisa baixar novamente.
> Total extra: **~6.5 GB**. Com os modelos T2V, o total fica ~14.5 GB.

In [ ]:
#@title 4b. Baixar Modelos I2V (~6.5GB extra)
#@markdown Execute esta célula apenas se quiser usar Image-to-Video.

import os

COMFYUI_PATH = "/content/ComfyUI"

# Modelos I2V adicionais
I2V_MODELS = [
    (
        "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/wan2.1_i2v_480p_1.3B_fp16.safetensors",
        f"{COMFYUI_PATH}/models/diffusion_models",
        "wan2.1_i2v_480p_1.3B_fp16.safetensors",
    ),
    (
        "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/clip_vision/clip_vision_h.safetensors",
        f"{COMFYUI_PATH}/models/clip_vision",
        "clip_vision_h.safetensors",
    ),
]

def format_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.1f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.1f} TB"

total_downloaded = 0

for url, dest_dir, filename in I2V_MODELS:
    filepath = os.path.join(dest_dir, filename)
    os.makedirs(dest_dir, exist_ok=True)

    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        print(f"✅ {filename} já existe ({format_size(size)}) — pulando download.")
    else:
        print(f"⬇️  Baixando {filename}...")
        !wget -q --show-progress -O "{filepath}" "{url}"
        if os.path.exists(filepath):
            size = os.path.getsize(filepath)
            total_downloaded += size
            print(f"   ✅ Concluído: {format_size(size)}")
        else:
            print(f"   ❌ ERRO: Download falhou para {filename}!")

# Resumo
print("\n" + "="*50)
print("📊 Modelos I2V:")
for url, dest_dir, filename in I2V_MODELS:
    filepath = os.path.join(dest_dir, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        print(f"   ✅ {filename}: {format_size(size)}")
    else:
        print(f"   ❌ {filename}: NÃO ENCONTRADO")
if total_downloaded > 0:
    print(f"\n   Total baixado: {format_size(total_downloaded)}")
print("="*50)
print("\n💡 Agora reinicie o ComfyUI (pare a célula 5 e execute-a novamente) para detectar os novos modelos.")

## 4c️⃣ Baixar Modelos IP-Adapter (Opcional — para manter rosto da imagem)
O IP-Adapter usa a imagem como **referência de estilo/rosto**, mantendo a semelhança facial no vídeo gerado.

| Ficheiro | Tamanho | Destino |
|---|---|---|
| `ip-adapter-plus-face_sdxl_vit-h.safetensors` | ~850 MB | `models/ipadapter/` |

> **Nota:** O `clip_vision_h.safetensors` da célula 4b é reutilizado pelo IP-Adapter — não precisa baixar outro.

In [ ]:
#@title 4c. Baixar Modelo IP-Adapter (~850MB)
#@markdown Necessário para usar IP-Adapter (manter rosto/estilo da imagem no vídeo).

import os

COMFYUI_PATH = "/content/ComfyUI"
IPADAPTER_DIR = f"{COMFYUI_PATH}/models/ipadapter"

os.makedirs(IPADAPTER_DIR, exist_ok=True)

IPA_MODEL = "ip-adapter-plus-face_sdxl_vit-h.safetensors"
IPA_URL = f"https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/{IPA_MODEL}"
IPA_PATH = os.path.join(IPADAPTER_DIR, IPA_MODEL)

if os.path.exists(IPA_PATH):
    size = os.path.getsize(IPA_PATH) / (1024**3)
    print(f"✅ {IPA_MODEL} já existe ({size:.2f} GB)")
else:
    print(f"⬇️  Baixando {IPA_MODEL}...")
    !wget --show-progress -O "{IPA_PATH}" "{IPA_URL}"
    if os.path.exists(IPA_PATH):
        size = os.path.getsize(IPA_PATH) / (1024**3)
        print(f"✅ IP-Adapter baixado: {size:.2f} GB")
    else:
        print(f"❌ Download falhou!")

# Verificar se clip_vision_h existe (necessário para IP-Adapter)
clip_path = f"{COMFYUI_PATH}/models/clip_vision/clip_vision_h.safetensors"
if os.path.exists(clip_path):
    print(f"✅ clip_vision_h.safetensors encontrado")
else:
    print(f"⚠️  clip_vision_h.safetensors NÃO encontrado — execute a célula 4b primeiro!")

print("\n💡 Reinicie o ComfyUI (pare e re-execute a célula 5) para detectar o novo modelo.")

## 5️⃣ Lançar ComfyUI (Modo Privado — Só API)

Esta célula lança o ComfyUI

> **Nota:** Esta célula ficará "a correr" enquanto o ComfyUI estiver ativo — é normal!
> Para gerar vídeos, execute as células 6 (T2V) ou 7 (I2V) **noutra célula**.

### Flags de otimização para T4:
| Flag | Função |
|---|---|
| `--lowvram` | Descarrega modelos para CPU quando não em uso |
| `--disable-smart-memory` | Evita cache de modelos que não cabem na VRAM |
| `--preview-method auto` | Preview em tempo real durante geração |


In [ ]:
#@title 5. Lançar ComfyUI com Túnel Público
import subprocess, threading, re, time, os

COMFYUI_PATH = "/content/ComfyUI"
PORT = 8188

# =============================================
# 1. Instalar Cloudflared
# =============================================
CLOUDFLARED_BIN = "/usr/local/bin/cloudflared"
if not os.path.exists(CLOUDFLARED_BIN):
    print("⬇️  Instalando Cloudflared...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O {CLOUDFLARED_BIN}
    !chmod +x {CLOUDFLARED_BIN}
    print("✅ Cloudflared instalado.")
else:
    print("✅ Cloudflared já instalado.")

# =============================================
# 2. Iniciar Túnel em Background
# =============================================
tunnel_url = None

def start_tunnel():
    global tunnel_url
    process = subprocess.Popen(
        [CLOUDFLARED_BIN, 'tunnel', '--url', f'http://127.0.0.1:{PORT}'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    for line in process.stderr:
        if '.trycloudflare.com' in line:
            match = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
            if match:
                tunnel_url = match.group()
                print("\n" + "="*60)
                print(f"🌐 ComfyUI URL PÚBLICO: {tunnel_url}")
                print("="*60)
                print("📋 Copie o URL acima e abra no browser!")
                print("   (Pode demorar alguns segundos para ficar acessível)\n")

tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()
print("🔄 Aguardando túnel Cloudflared...")
time.sleep(5)

if tunnel_url is None:
    print("⏳ Túnel ainda a iniciar... O URL aparecerá em breve abaixo.")

# =============================================
# 3. Lançar ComfyUI (bloqueia esta célula)
# =============================================
print(f"\n🚀 Iniciando ComfyUI na porta {PORT}...")
print("   (Esta célula ficará em execução — é normal!)\n")

%cd {COMFYUI_PATH}
!python main.py \
    --listen 0.0.0.0 \
    --port {PORT} \
    --lowvram \
    --disable-smart-memory \
    --preview-method auto

## 6️⃣ Gerar Vídeo via API (Bónus — Opcional)

> **IMPORTANTE:** Só execute esta célula **DEPOIS** da célula 5 estar a correr e o ComfyUI estar acessível.

Esta célula envia um workflow completo via API do ComfyUI para gerar um vídeo automaticamente, sem precisar montar o workflow manualmente na interface.

**Parâmetros padrão:**
- Resolução: 480×320 (otimizado para T4)
- Frames: 33 (~2 segundos de vídeo)
- Steps: 25 | Sampler: euler | Scheduler: normal | CFG: 6.0
- Output: **MP4** via VideoHelperSuite (`VideoCombine`) + WEBP como backup

> Edite a variável `PROMPT` abaixo para definir o que deseja gerar.
> Para vídeos curtos (33-61 frames), o sampler **Euler** com scheduler **Normal** funciona melhor.

In [ ]:
#@title 6. Gerar Vídeo Automaticamente via API
#@markdown Edite o prompt abaixo e execute esta célula (ComfyUI deve estar a correr na célula 5).

PROMPT = "A golden retriever running through a sunlit meadow with wildflowers, cinematic lighting, slow motion, 4K quality" #@param {type:"string"}

#@markdown ---
#@markdown **Configurações de vídeo:**
WIDTH = 480    #@param {type:"integer"}
HEIGHT = 320   #@param {type:"integer"}
FRAMES = 33    #@param {type:"integer"}
STEPS = 25     #@param {type:"integer"}
CFG = 6.0      #@param {type:"number"}
SEED = -1      #@param {type:"integer"}
SAMPLER = "euler"  #@param ["euler", "euler_ancestral", "dpmpp_2m", "uni_pc"]
SCHEDULER = "normal" #@param ["normal", "simple", "karras", "sgm_uniform"]
OUTPUT_FORMAT = "mp4" #@param ["mp4", "webp", "gif"]
FPS = 16       #@param {type:"integer"}
#@markdown <small>SEED = -1 para aleatório. Frames deve ser 4n+1 (5,9,13,17,21,25,29,33,49,65).</small>

import json, requests, random, time, os, glob
from IPython.display import display, Image, HTML, Video

COMFYUI_URL = "http://127.0.0.1:8188"

# Seed aleatório se -1
if SEED == -1:
    SEED = random.randint(0, 2**32 - 1)
print(f"🎲 Seed: {SEED}")

# =============================================
# Workflow JSON da API do ComfyUI
# =============================================
# Nodes do workflow:
# 1: UNETLoader         — carrega diffusion model Wan2.1
# 2: CLIPLoader          — carrega text encoder UMT5-XXL (FP8)
# 3: VAELoader           — carrega VAE
# 4: CLIPTextEncode      — prompt positivo
# 5: CLIPTextEncode      — prompt negativo (vazio)
# 6: EmptyWanLatentVideo  — cria latent com dimensões do vídeo
# 7: KSampler            — amostragem/geração
# 8: VAEDecode           — decodifica latent → frames
# 9: VHS_VideoCombine    — salva como MP4/GIF (VideoHelperSuite)
# 10: SaveAnimatedWEBP   — backup em WEBP (nativo ComfyUI)

workflow = {
    "1": {
        "class_type": "UNETLoader",
        "inputs": {
            "unet_name": "wan2.1_t2v_1.3B_fp16.safetensors",
            "weight_dtype": "default"
        }
    },
    "2": {
        "class_type": "CLIPLoader",
        "inputs": {
            "clip_name": "umt5_xxl_fp8_e4m3fn_scaled.safetensors",
            "type": "wan"
        }
    },
    "3": {
        "class_type": "VAELoader",
        "inputs": {
            "vae_name": "wan_2.1_vae.safetensors"
        }
    },
    "4": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": PROMPT,
            "clip": ["2", 0]
        }
    },
    "5": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": "",
            "clip": ["2", 0]
        }
    },
    "6": {
        "class_type": "EmptyWanLatentVideo",
        "inputs": {
            "width": WIDTH,
            "height": HEIGHT,
            "length": FRAMES,
            "batch_size": 1
        }
    },
    "7": {
        "class_type": "KSampler",
        "inputs": {
            "seed": SEED,
            "steps": STEPS,
            "cfg": CFG,
            "sampler_name": SAMPLER,
            "scheduler": SCHEDULER,
            "denoise": 1.0,
            "model": ["1", 0],
            "positive": ["4", 0],
            "negative": ["5", 0],
            "latent_image": ["6", 0]
        }
    },
    "8": {
        "class_type": "VAEDecode",
        "inputs": {
            "samples": ["7", 0],
            "vae": ["3", 0]
        }
    },
    # VideoHelperSuite — salva como MP4/GIF com ffmpeg
    "9": {
        "class_type": "VHS_VideoCombine",
        "inputs": {
            "frame_rate": FPS,
            "loop_count": 0,
            "filename_prefix": "wan21_video",
            "format": f"video/{OUTPUT_FORMAT}",
            "save_output": True,
            "images": ["8", 0]
        }
    },
    # Backup: WEBP nativo do ComfyUI (sempre salva)
    "10": {
        "class_type": "SaveAnimatedWEBP",
        "inputs": {
            "filename_prefix": "wan21_backup",
            "fps": FPS,
            "lossless": False,
            "quality": 85,
            "method": "default",
            "images": ["8", 0]
        }
    }
}

# =============================================
# Enviar Workflow via API
# =============================================
print(f"\n📤 Enviando workflow para ComfyUI...")
print(f"   Prompt: {PROMPT[:80]}{'...' if len(PROMPT) > 80 else ''}")
print(f"   Resolução: {WIDTH}×{HEIGHT} | Frames: {FRAMES} (~{FRAMES/FPS:.1f}s)")
print(f"   Steps: {STEPS} | Sampler: {SAMPLER} | Scheduler: {SCHEDULER} | CFG: {CFG}")
print(f"   Formato: {OUTPUT_FORMAT.upper()} | FPS: {FPS}")

try:
    response = requests.post(
        f"{COMFYUI_URL}/prompt",
        json={"prompt": workflow},
        timeout=10
    )
    if response.status_code == 200:
        result = response.json()
        prompt_id = result.get("prompt_id", "unknown")
        print(f"   ✅ Workflow aceite! Prompt ID: {prompt_id}")
    else:
        print(f"   ❌ Erro: HTTP {response.status_code}")
        print(f"   {response.text[:500]}")
        # Se VHS_VideoCombine não está disponível, tentar sem ele
        if "VHS_VideoCombine" in response.text:
            print("\n   ⚠️  VideoHelperSuite não encontrado. Tentando workflow sem MP4...")
            del workflow["9"]
            response = requests.post(f"{COMFYUI_URL}/prompt", json={"prompt": workflow}, timeout=10)
            if response.status_code == 200:
                result = response.json()
                prompt_id = result.get("prompt_id", "unknown")
                OUTPUT_FORMAT = "webp"
                print(f"   ✅ Workflow (WEBP only) aceite! Prompt ID: {prompt_id}")
            else:
                print(f"   ❌ Falhou novamente: {response.text[:300]}")
                raise SystemExit
        else:
            raise SystemExit
except requests.exceptions.ConnectionError:
    print("   ❌ ComfyUI não está a responder! Verifique se a célula 5 está em execução.")
    raise SystemExit

# =============================================
# Monitorizar Progresso
# =============================================
print(f"\n⏳ A gerar vídeo... (estimativa: 10-30 min no T4)")
print(f"   Acompanhe o progresso em tempo real no ComfyUI (URL da célula 5).\n")

OUTPUT_DIR = "/content/ComfyUI/output"
start_time = time.time()
completed = False

# Ficheiros existentes antes da geração
existing_mp4 = set(glob.glob(f"{OUTPUT_DIR}/wan21_video*.mp4"))
existing_webp = set(glob.glob(f"{OUTPUT_DIR}/wan21_video*.webp") |
                    glob.glob(f"{OUTPUT_DIR}/wan21_backup*.webp"))
existing_gif = set(glob.glob(f"{OUTPUT_DIR}/wan21_video*.gif"))
existing_all = existing_mp4 | existing_webp | existing_gif

while not completed:
    time.sleep(10)

    # Verificar se o prompt foi concluído
    try:
        history = requests.get(f"{COMFYUI_URL}/history/{prompt_id}", timeout=5).json()
        if prompt_id in history:
            outputs = history[prompt_id].get("outputs", {})
            if outputs:
                completed = True
                break
    except:
        pass

    # Feedback de tempo
    elapsed = time.time() - start_time
    mins = int(elapsed // 60)
    secs = int(elapsed % 60)
    print(f"   ⏱️  {mins}m {secs}s decorridos...", end="\r")

    # Timeout após 60 min
    if elapsed > 3600:
        print("\n   ⚠️  Timeout (60 min). Verifique o ComfyUI para erros.")
        break

# =============================================
# Resultado
# =============================================
elapsed = time.time() - start_time
mins = int(elapsed // 60)
secs = int(elapsed % 60)

# Encontrar novos ficheiros de vídeo
current_all = set(
    glob.glob(f"{OUTPUT_DIR}/wan21_video*.*") +
    glob.glob(f"{OUTPUT_DIR}/wan21_backup*.*")
)
new_files = current_all - existing_all

if new_files:
    # Prioridade: MP4 > GIF > WEBP
    mp4s = [f for f in new_files if f.endswith('.mp4')]
    gifs = [f for f in new_files if f.endswith('.gif')]
    webps = [f for f in new_files if f.endswith('.webp')]
    best = mp4s or gifs or webps
    latest = max(best, key=os.path.getmtime) if best else max(new_files, key=os.path.getmtime)

    size = os.path.getsize(latest) / (1024 * 1024)
    ext = os.path.splitext(latest)[1].upper()

    print(f"\n{'='*60}")
    print(f"🎬 VÍDEO GERADO COM SUCESSO!")
    print(f"   Ficheiro: {latest}")
    print(f"   Formato: {ext} | Tamanho: {size:.1f} MB")
    print(f"   Tempo: {mins}m {secs}s")
    print(f"   Seed: {SEED}")
    print(f"{'='*60}")

    # Listar todos os ficheiros gerados
    if len(new_files) > 1:
        print(f"\n   📁 Todos os ficheiros gerados:")
        for f in sorted(new_files):
            s = os.path.getsize(f) / (1024 * 1024)
            print(f"      • {os.path.basename(f)} ({s:.1f} MB)")

    # Exibir inline
    print()
    if latest.endswith('.mp4'):
        display(Video(latest, embed=True, width=WIDTH))
    elif latest.endswith('.webp') or latest.endswith('.gif'):
        display(Image(filename=latest))
else:
    print(f"\n⚠️  Não foram encontrados novos ficheiros em {OUTPUT_DIR}")
    print(f"   Verifique o ComfyUI para possíveis erros.")
    print(f"   Ficheiros existentes:")
    for f in sorted(glob.glob(f"{OUTPUT_DIR}/*")):
        print(f"      • {os.path.basename(f)}")

## 7️⃣ Gerar Vídeo a partir de Imagem (I2V) — via API

> **IMPORTANTE:** Requer os modelos da célula 4b. O ComfyUI deve estar a correr (célula 5).

Workflow: Carrega uma imagem de input → anima-a com Wan2.1-I2V-1.3B → salva como MP4.

**Como funciona:**
1. Faz upload da imagem para o ComfyUI
2. O CLIP Vision codifica a imagem
3. O modelo I2V gera vídeo a partir da imagem + prompt de texto
4. Resultado salvo como MP4/WEBP

In [ ]:
#@title 7. Image-to-Video (Wan2.1 I2V + IP-Adapter) via API
#@markdown Faça upload de uma imagem e gere um vídeo a partir dela.

#@markdown **Imagem de input** (caminho local no Colab ou URL):
IMAGE_PATH = "" #@param {type:"string"}
#@markdown <small>Deixe vazio para fazer upload interativo. Ou cole um caminho como `/content/minha_imagem.png` ou URL `https://...`</small>

PROMPT_I2V = "cinematic smooth motion, high quality, detailed animation, realistic skin textures" #@param {type:"string"}
NEGATIVE_PROMPT_I2V = "(worst quality:2), (low quality:2), (normal quality:2), lowres, blurry, (distorted anatomy:1.4), (extra limbs:1.4), (bad proportions:1.4), watermark, text, error, blurry face, (plastic skin:1.3), cartoon, anime, 3d render, cgi, (cloned face:1.2), missing fingers, extra fingers" #@param {type:"string"}

#@markdown ---
#@markdown **Configurações de vídeo:**
WIDTH_I2V = 480    #@param {type:"integer"}
HEIGHT_I2V = 320   #@param {type:"integer"}
FRAMES_I2V = 33    #@param {type:"integer"}
STEPS_I2V = 20     #@param {type:"integer"}
CFG_I2V = 6.0      #@param {type:"number"}
SEED_I2V = -1      #@param {type:"integer"}
SAMPLER_I2V = "uni_pc"  #@param ["euler", "euler_ancestral", "dpmpp_2m", "uni_pc"]
SCHEDULER_I2V = "simple" #@param ["normal", "simple", "karras", "sgm_uniform"]
OUTPUT_FORMAT_I2V = "mp4" #@param ["mp4", "webp", "gif"]
FPS_I2V = 16       #@param {type:"integer"}

#@markdown ---
#@markdown **IP-Adapter (mantém rosto/estilo da imagem):**
USE_IPADAPTER = True  #@param {type:"boolean"}
IPA_WEIGHT = 0.7      #@param {type:"number"}
IPA_WEIGHT_TYPE = "linear" #@param ["linear", "ease in", "ease out", "ease in-out", "reverse in-out", "weak input", "weak output", "weak middle", "strong middle"]
#@markdown <small>IPA_WEIGHT: 0.5-0.8 recomendado (mais alto = mais fiel à imagem, menos movimento).</small>

import json, requests, random, time, os, glob, shutil
from IPython.display import display, Image as IPImage, HTML, Video
from PIL import Image as PILImage
import io

COMFYUI_URL = "http://127.0.0.1:8188"
COMFYUI_INPUT = "/content/ComfyUI/input"

# =============================================
# 1. Obter/Upload da Imagem
# =============================================
os.makedirs(COMFYUI_INPUT, exist_ok=True)
input_filename = None

if IMAGE_PATH.strip() == "":
    # Upload interativo no Colab
    print("📤 Selecione uma imagem para upload...")
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        fname = list(uploaded.keys())[0]
        dest = os.path.join(COMFYUI_INPUT, fname)
        with open(dest, 'wb') as f:
            f.write(uploaded[fname])
        input_filename = fname
        print(f"   ✅ Imagem salva: {dest}")
    else:
        raise SystemExit("❌ Nenhuma imagem foi selecionada.")
elif IMAGE_PATH.startswith("http"):
    # Baixar de URL
    print(f"⬇️  Baixando imagem de URL...")
    import urllib.request
    fname = os.path.basename(IMAGE_PATH).split("?")[0]
    if not fname or '.' not in fname:
        fname = "input_image.png"
    dest = os.path.join(COMFYUI_INPUT, fname)
    urllib.request.urlretrieve(IMAGE_PATH, dest)
    input_filename = fname
    print(f"   ✅ Imagem salva: {dest}")
else:
    # Caminho local
    if os.path.exists(IMAGE_PATH):
        fname = os.path.basename(IMAGE_PATH)
        dest = os.path.join(COMFYUI_INPUT, fname)
        if os.path.abspath(IMAGE_PATH) != os.path.abspath(dest):
            shutil.copy2(IMAGE_PATH, dest)
        input_filename = fname
        print(f"   ✅ Imagem copiada: {dest}")
    else:
        raise SystemExit(f"❌ Ficheiro não encontrado: {IMAGE_PATH}")

# Mostrar imagem de input
print(f"\n🖼️  Imagem de input: {input_filename}")
img = PILImage.open(os.path.join(COMFYUI_INPUT, input_filename))
print(f"   Dimensões originais: {img.width}×{img.height}")
display(IPImage(filename=os.path.join(COMFYUI_INPUT, input_filename), width=300))

# =============================================
# 2. Upload da imagem para ComfyUI via API
# =============================================
print(f"\n📤 Enviando imagem para ComfyUI...")
with open(os.path.join(COMFYUI_INPUT, input_filename), 'rb') as f:
    resp = requests.post(
        f"{COMFYUI_URL}/upload/image",
        files={"image": (input_filename, f, "image/png")},
        data={"overwrite": "true"}
    )
if resp.status_code == 200:
    upload_result = resp.json()
    comfyui_image_name = upload_result.get("name", input_filename)
    print(f"   ✅ Upload OK: {comfyui_image_name}")
else:
    print(f"   ❌ Erro upload: {resp.status_code} — {resp.text[:300]}")
    raise SystemExit

# =============================================
# 3. Workflow I2V + IP-Adapter
# =============================================
if SEED_I2V == -1:
    SEED_I2V = random.randint(0, 2**32 - 1)
print(f"🎲 Seed: {SEED_I2V}")
print(f"🖼️  IP-Adapter: {'ACTIVO (peso: ' + str(IPA_WEIGHT) + ')' if USE_IPADAPTER else 'DESACTIVADO'}")

# Nodes base:
# 1: UNETLoader           — modelo I2V (1.3B)
# 2: CLIPLoader            — text encoder UMT5-XXL
# 3: VAELoader             — VAE
# 4: CLIPVisionLoader      — CLIP Vision H
# 5: CLIPTextEncode        — prompt positivo
# 6: CLIPTextEncode        — prompt negativo
# 7: LoadImage             — imagem de input
# 8: CLIPVisionEncode      — codifica imagem com CLIP Vision
# 9: WanImageToVideo       — gera latent I2V
# 10: KSampler             — amostragem
# 11: VAEDecode            — decode → frames
# 12: VHS_VideoCombine     — MP4/GIF
# 13: SaveAnimatedWEBP     — backup WEBP
# 14: IPAdapterModelLoader — carrega modelo IP-Adapter (se activo)
# 15: IPAdapterAdvanced    — aplica IP-Adapter ao modelo (se activo)

workflow_i2v = {
    "1": {
        "class_type": "UNETLoader",
        "inputs": {
            "unet_name": "wan2.1_i2v_480p_1.3B_fp16.safetensors",
            "weight_dtype": "default"
        }
    },
    "2": {
        "class_type": "CLIPLoader",
        "inputs": {
            "clip_name": "umt5_xxl_fp8_e4m3fn_scaled.safetensors",
            "type": "wan"
        }
    },
    "3": {
        "class_type": "VAELoader",
        "inputs": {
            "vae_name": "wan_2.1_vae.safetensors"
        }
    },
    "4": {
        "class_type": "CLIPVisionLoader",
        "inputs": {
            "clip_name": "clip_vision_h.safetensors"
        }
    },
    "5": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": PROMPT_I2V,
            "clip": ["2", 0]
        }
    },
    "6": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "text": NEGATIVE_PROMPT_I2V,
            "clip": ["2", 0]
        }
    },
    "7": {
        "class_type": "LoadImage",
        "inputs": {
            "image": comfyui_image_name
        }
    },
    "8": {
        "class_type": "CLIPVisionEncode",
        "inputs": {
            "clip_vision": ["4", 0],
            "image": ["7", 0]
        }
    },
    "9": {
        "class_type": "WanImageToVideo",
        "inputs": {
            "width": WIDTH_I2V,
            "height": HEIGHT_I2V,
            "length": FRAMES_I2V,
            "batch_size": 1,
            "positive": ["5", 0],
            "negative": ["6", 0],
            "vae": ["3", 0],
            "clip_vision_output": ["8", 0],
            "start_image": ["7", 0]
        }
    },
    "10": {
        "class_type": "KSampler",
        "inputs": {
            "seed": SEED_I2V,
            "steps": STEPS_I2V,
            "cfg": CFG_I2V,
            "sampler_name": SAMPLER_I2V,
            "scheduler": SCHEDULER_I2V,
            "denoise": 1.0,
            "model": ["1", 0],  # será substituído pelo IP-Adapter se activo
            "positive": ["9", 0],
            "negative": ["9", 1],
            "latent_image": ["9", 2]
        }
    },
    "11": {
        "class_type": "VAEDecode",
        "inputs": {
            "samples": ["10", 0],
            "vae": ["3", 0]
        }
    },
    "12": {
        "class_type": "VHS_VideoCombine",
        "inputs": {
            "frame_rate": FPS_I2V,
            "loop_count": 0,
            "filename_prefix": "wan21_i2v",
            "format": f"video/{OUTPUT_FORMAT_I2V}",
            "save_output": True,
            "images": ["11", 0]
        }
    },
    "13": {
        "class_type": "SaveAnimatedWEBP",
        "inputs": {
            "filename_prefix": "wan21_i2v_backup",
            "fps": FPS_I2V,
            "lossless": False,
            "quality": 85,
            "method": "default",
            "images": ["11", 0]
        }
    }
}

# Adicionar IP-Adapter se activo
if USE_IPADAPTER:
    # Node 14: Carregar modelo IP-Adapter
    workflow_i2v["14"] = {
        "class_type": "IPAdapterModelLoader",
        "inputs": {
            "ipadapter_file": "ip-adapter-plus-face_sdxl_vit-h.safetensors"
        }
    }
    # Node 15: Aplicar IP-Adapter ao modelo
    workflow_i2v["15"] = {
        "class_type": "IPAdapterAdvanced",
        "inputs": {
            "weight": IPA_WEIGHT,
            "weight_type": IPA_WEIGHT_TYPE,
            "combine_embeds": "concat",
            "start_at": 0.0,
            "end_at": 1.0,
            "embeds_scaling": "V only",
            "model": ["1", 0],
            "ipadapter": ["14", 0],
            "image": ["7", 0],
            "clip_vision": ["4", 0]
        }
    }
    # KSampler usa o modelo com IP-Adapter em vez do modelo direto
    workflow_i2v["10"]["inputs"]["model"] = ["15", 0]

# =============================================
# 4. Enviar Workflow
# =============================================
print(f"\n📤 Enviando workflow I2V para ComfyUI...")
print(f"   Imagem: {comfyui_image_name}")
print(f"   Prompt: {PROMPT_I2V[:80]}{'...' if len(PROMPT_I2V) > 80 else ''}")
print(f"   Resolução: {WIDTH_I2V}×{HEIGHT_I2V} | Frames: {FRAMES_I2V} (~{FRAMES_I2V/FPS_I2V:.1f}s)")
print(f"   Steps: {STEPS_I2V} | Sampler: {SAMPLER_I2V} | Scheduler: {SCHEDULER_I2V} | CFG: {CFG_I2V}")
if USE_IPADAPTER:
    print(f"   IP-Adapter: peso {IPA_WEIGHT} | tipo {IPA_WEIGHT_TYPE}")

try:
    response = requests.post(
        f"{COMFYUI_URL}/prompt",
        json={"prompt": workflow_i2v},
        timeout=10
    )
    if response.status_code == 200:
        result = response.json()
        prompt_id = result.get("prompt_id", "unknown")
        print(f"   ✅ Workflow I2V aceite! Prompt ID: {prompt_id}")
    else:
        print(f"   ❌ Erro: HTTP {response.status_code}")
        error_text = response.text[:500]
        print(f"   {error_text}")

        # Fallback: se IP-Adapter falha, tentar sem ele
        if USE_IPADAPTER and ("IPAdapter" in error_text or "ipadapter" in error_text.lower()):
            print("\n   ⚠️  IP-Adapter falhou. Tentando sem IP-Adapter...")
            del workflow_i2v["14"]
            del workflow_i2v["15"]
            workflow_i2v["10"]["inputs"]["model"] = ["1", 0]
            response = requests.post(f"{COMFYUI_URL}/prompt", json={"prompt": workflow_i2v}, timeout=10)
            if response.status_code == 200:
                result = response.json()
                prompt_id = result.get("prompt_id", "unknown")
                print(f"   ✅ Workflow I2V (sem IP-Adapter) aceite! Prompt ID: {prompt_id}")
            else:
                print(f"   ❌ Falhou: {response.text[:300]}")
                raise SystemExit

        # Fallback: remover VHS_VideoCombine se não disponível
        elif "VHS_VideoCombine" in error_text:
            print("\n   ⚠️  VideoHelperSuite não encontrado. Tentando sem MP4...")
            del workflow_i2v["12"]
            response = requests.post(f"{COMFYUI_URL}/prompt", json={"prompt": workflow_i2v}, timeout=10)
            if response.status_code == 200:
                result = response.json()
                prompt_id = result.get("prompt_id", "unknown")
                OUTPUT_FORMAT_I2V = "webp"
                print(f"   ✅ Workflow I2V (WEBP) aceite! Prompt ID: {prompt_id}")
            else:
                print(f"   ❌ Falhou: {response.text[:300]}")
                raise SystemExit

        # Modelos I2V não encontrados
        elif "wan2.1_i2v" in error_text or "clip_vision_h" in error_text:
            print("\n   ❌ Modelos I2V não encontrados!")
            print("   Execute a célula 4b primeiro para baixar os modelos I2V.")
            print("   Depois reinicie o ComfyUI (pare e re-execute a célula 5).")
            raise SystemExit
        else:
            raise SystemExit
except requests.exceptions.ConnectionError:
    print("   ❌ ComfyUI não está a responder! Verifique se a célula 5 está em execução.")
    raise SystemExit

# =============================================
# 5. Monitorizar Progresso
# =============================================
print(f"\n⏳ A gerar vídeo I2V... (estimativa: 10-30 min no T4)")

OUTPUT_DIR = "/content/ComfyUI/output"
start_time = time.time()
completed = False

existing_all = set(
    glob.glob(f"{OUTPUT_DIR}/wan21_i2v*.*") +
    glob.glob(f"{OUTPUT_DIR}/wan21_i2v_backup*.*")
)

while not completed:
    time.sleep(10)
    try:
        history = requests.get(f"{COMFYUI_URL}/history/{prompt_id}", timeout=5).json()
        if prompt_id in history:
            outputs = history[prompt_id].get("outputs", {})
            if outputs:
                completed = True
                break
    except:
        pass

    elapsed = time.time() - start_time
    mins = int(elapsed // 60)
    secs = int(elapsed % 60)
    print(f"   ⏱️  {mins}m {secs}s decorridos...", end="\r")

    if elapsed > 3600:
        print("\n   ⚠️  Timeout (60 min).")
        break

# =============================================
# 6. Resultado
# =============================================
elapsed = time.time() - start_time
mins = int(elapsed // 60)
secs = int(elapsed % 60)

current_all = set(
    glob.glob(f"{OUTPUT_DIR}/wan21_i2v*.*") +
    glob.glob(f"{OUTPUT_DIR}/wan21_i2v_backup*.*")
)
new_files = current_all - existing_all

if new_files:
    mp4s = [f for f in new_files if f.endswith('.mp4')]
    gifs = [f for f in new_files if f.endswith('.gif')]
    webps = [f for f in new_files if f.endswith('.webp')]
    best = mp4s or gifs or webps
    latest = max(best, key=os.path.getmtime) if best else max(new_files, key=os.path.getmtime)

    size = os.path.getsize(latest) / (1024 * 1024)
    ext = os.path.splitext(latest)[1].upper()

    print(f"\n{'='*60}")
    print(f"🎬 VÍDEO I2V GERADO COM SUCESSO!")
    print(f"   Ficheiro: {latest}")
    print(f"   Formato: {ext} | Tamanho: {size:.1f} MB")
    print(f"   Tempo: {mins}m {secs}s")
    print(f"   Seed: {SEED_I2V}")
    if USE_IPADAPTER:
        print(f"   IP-Adapter: peso {IPA_WEIGHT}")
    print(f"{'='*60}")

    if len(new_files) > 1:
        print(f"\n   📁 Todos os ficheiros:")
        for f in sorted(new_files):
            s = os.path.getsize(f) / (1024 * 1024)
            print(f"      • {os.path.basename(f)} ({s:.1f} MB)")

    print()
    if latest.endswith('.mp4'):
        display(Video(latest, embed=True, width=WIDTH_I2V))
    elif latest.endswith('.webp') or latest.endswith('.gif'):
        display(IPImage(filename=latest))
else:
    print(f"\n⚠️  Não foram encontrados novos ficheiros.")
    print(f"   Verifique o ComfyUI para erros.")

---

## 📖 Guia Rápido

### Como gerar vídeos:
1. Execute as células **1 → 5** em ordem
2. Para **Texto → Vídeo**: execute a **célula 6** (edite o `PROMPT` antes)
3. Para **Imagem → Vídeo**: execute a célula **4b** (modelos extra) e depois a **célula 7**

### Configurações recomendadas para T4:
| Parâmetro | Valor | Notas |
|---|---|---|
| Resolução | 480×320 ou 512×320 | Não ultrapassar no T4 |
| Frames | 33 (2s) ou 49 (3s) | Deve ser `4n+1` |
| Steps | 25-30 | 25 é bom equilíbrio speed/quality |
| Sampler | `euler` ou `uni_pc` | UniPC mais rápido, Euler melhor qualidade |
| Scheduler | `normal` ou `simple` | Normal é recomendado para Wan2.1 |
| CFG | 5.0-7.0 | 6.0 é o sweet spot |
| FPS output | 16 | Wan2.1 treinou com 16fps — não alterar |

### Dicas:
- **Frames devem ser `4n+1`**: 5, 9, 13, 17, 21, 25, 29, 33, 49, 65, 81
- **Não ultrapasse 480×320** no T4 — resoluções maiores causam OOM
- **Prompts em inglês** funcionam melhor com Wan2.1
- **Geração demora 10-30 min** por clip no T4 — é normal!
- Os vídeos são salvos em `/content/ComfyUI/output/`
- **`--disable-smart-memory`** evita cache agressivo que causa OOM no T4
- Se o Text Encoder causar OOM, experimente o GGUF Q4_K_M (~2.5GB) via ComfyUI-GGUF
- **Modo privado**: O ComfyUI só aceita conexões de localhost — ninguém de fora acede